#### Logistic Regression using conjugate gradient Descent

In [2]:
import os
import sys
import timeit
import numpy as np
import theano
import theano.tensor as T

the following codes will import the necessary python packages for this program

In [11]:
class logisticregression(object):
    def __init__(self,input, n_in, n_out):
        self.theta = theano.shared(value=numpy.zeros(n_in * n_out + n_out, dtype = theano.config.floatX), name='theta', borrow = True)
        self.W = self.theta[0:n_in * n_out].reshape((n_in, n_out))
        self.b = self.theta[n_in * n_out:n_in * n_out+n_out]
        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W)+self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis=1)
        self.input = input
    def negative_log_likelihood(self, y ):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]),y])
    def errors(self, y):
        if y.ndim != self.y_pred.ndim:
            raise TypeError('y should have the same shape as the self.y_pred', 'y',y.type, 'y_pred',self.y_pred.type)
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y ))
        else:
            raise NotImplementedError()
            
    def load_data(dataset):
        data_dir, data_file = os.path.split(dataset)
        if data_dir == "" and not os.path.isfile(dataset):
            new_path = os.path.join(os.path.split(__file__)[0],"..","data", dataset)
            if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
                dataset= new_path
                
        if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
            from six.moves import urllib
            origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
                )
            print ('Downloading data from %s '%origin)
            urllib.request.urlretrieve(origin, dataset)
            
        print ('-----loading datasets')
        with gzip.open(dataset, 'rb') as f:
            try:
                train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
            except:
                train_set, valid_set, test_set = pickle.load(f)
                
    def cg_optimization_mnist(n_epochs = 50, mnist_pkl_gz='mnist.pkl.gz'):
        datasets = load_data(mnist_pkl.gz)
        train_set_x, train_set_y = datasets[0]
        valid_set_x, valid_set_y = datasets[1]
        test_set_x, test_set_y = datasets[2]
        
        batch_size = 600
        n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
        n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
        n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size
        
        n_in = 28 * 28
        n_out = 10
        
        print '.... Building Model'
        minibatch_offset = T.lscalar()
        x = T.matrix()
        y = T.ivector()
    
        classifier = logisticregression(input=x,n_in=28*28, n_out=10)
        cost = classifier.negative_log_likelihood(y).mean()
        test_model = theano.function(
        [minibatch_offset],
        classifier.errors(y),
        givens = {
                x:test_set_x[minibatch_offset:minibatch_offset + batch_size],
                y:test_set_x[minibatch_offset:minibatch_offset + batch_size]
                },
        name = "test")
        validate_model = theano.function(
        [minibatch_offset],
        classifier.errors(y),
            givens = {
                x:valid_set_x[minibatch_offset:minibatch_offset +batch_size],
                y:valid_set_y[minibatch_offset:minibatch_offset + batch_size]
            },name = "validate"
        )
        batch_cost = theano.function(
        [minibatch_offset],
        cost, 
        givens = {
                x:train_set_x[minibatch_offset:minibatch_offset + batch_size],
                y:train_set_x[minibatch_offset:minibatch_offset + batch_size]
                
            }, name = 'batch_cost')
        batch_grad = theano.function(
        [minibatch_offset],
        T.grad(cost, classifier.theta),
            givens = 
            {x:train_set[minibatch_offset: minibatch_offset + batch_size],
            y: train_set[minibatch_offset: minibatch_offset + batch_size]},
            name = "batch_grad"
        )
        
    def train_fn(theta_value):
        classifier.theta.set_value(theta_value, borrow=True)
        train_losses = [batch_cost(i * batch_size) for i in range(n_train_batches)]
        return numpy.mean(train_losses)
    
    def train_fn_grad(theta_values):
        classifier.theta.set_value(theta_value, borrow=True)
        grad = batch_grad(0)
        for i in range(1, n_train_batches):
            grad += batch_grad(i * batch_size)
        return grad / n_train_batches
    
    validation_scores = [numpy.inf, 0]
    
    def callback(theta_value):
        classifier.theta.set_value(theta_value, borrow=True)
        validation_losses = [validate_model(i * batch_size) for i in range(n_valid_batches)]
        this_validation_loss = np.mean(validation_losses)
        print('validation error %f %%'%(this_validation_loss * 100.,))
        
        if this_validation_loss < validation_scores[0]:
            validation_scores[0] = this_validation_loss
            test_losses = [test_model(i * batch_size)
                          for i in range(n_test_batches)]
            validation_scores[1] = numpy.mean(test_losses)
            
    import scipy.optimize
    print ("Optimising using the scipy.optimize.fmin_cg")
    start_w_b = timeit.default_timer()
    best_w_b = scipy.optimize.fmin_cg(
    f = train_fn,
        x0 = numpy.zeros((n_in + 1)* n_out, dtype=x.dtype),
        fprime = train_fn_grad,
        callback=callback,
        disp = 0,
        maxiter = n_epochs
        
    )
    end_time = timeit.default_timer()
    print(
    ('Optimising Complete with the best Validation score of %f %%, with '
        'test performance %f %%'
    )% (validation_scores[0] * 100., validation_scores[1] * 100.)
        
    )
    print >> sys.stderr, ('the code for the file ' + os.path.split(__file__)[1] + 'ran for %.1fs'%((end_time - start)))

Optimising using the scipy.optimize.fmin_cg


NameError: name 'n_in' is not defined

In [ ]:
if __name__ = '__main__':
    cg_optimization_mnist()